# Imports

In [ ]:
import pandas as pd
from datetime import datetime as dt
from pathlib import Path
from hbt_tools import messdatenbank as mdb
from plotly import graph_objects as go
from mikeio1d import Res1D
from mikeio1d.query import QueryDataReach

# Functions

In [ ]:
def periods_to_dt(periods, date_format):
    periods_out = []
    for i in range(len(periods)):
        dt1 = dt.strptime(periods[i][0], date_format)
        dt2 = dt.strptime(periods[i][1], date_format)
        periods_out.append((dt1, dt2))

    return periods_out
#-------------------------------------------------------------------------------

def ts_filter_date(ts, date_from, date_to):
    ts = ts.copy()
    ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts
#-------------------------------------------------------------------------------

def get_data_from_muids(df_res1d, quantity, muids):
    data = []
    for muid in muids:
        query = f"{quantity}{muid}"
        mask = df_res1d.columns.str.startswith(query)
        df = df_res1d.loc[:,mask].copy()
        data.append(df)
    data = pd.concat(data, axis=1)

    return data
#-------------------------------------------------------------------------------

def compute_tot_sum(ts):
    ts['TOT'] = 0
    for col in ts.columns[0:-1]:
        ts['TOT'] += ts[col]
#-------------------------------------------------------------------------------

# Parameters

In [ ]:
pumpwerke = ['PW Klosterweg', 'PW Mülibach', 'PW Mülenen']

info_pw = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\5 Fremdwasser\Pumpwerke\Pumpwerke_Angaben.xlsx")
info_pw = pd.read_excel(info_pw)
info_pw = info_pw[info_pw['Bezeichnung'].isin(pumpwerke)]

data_pw = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\5 Fremdwasser\Pumpwerke\Daten\220722_10.56Uhr_Niveau_PW_1-jahr.csv")

path_res1d = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\Mike+ Modell\04_Modell"
    r"\IST-Zustand\Richterswil_IST_m1d - Result Files\Richterswil_IST_wae_230725Default_Network_HD.res1d")

date_format = '%d.%m.%Y %H:%M'
rain_periods = [
    ('25.04.2022 01:10', '25.04.2022 03:20'),
    ('16.08.2021 13:00', '16.08.2021 15:00'),
    ('24.04.2022 22:00', '25.04.2022 02:00'),
    ('24.07.2021 16:10', '24.07.2021 18:00')
]
periods_to_dt(rain_periods, date_format)

date_from = dt(2021, 7, 1, 0, 0)
date_to = dt(2022, 7, 1, 23, 59)

# Read Data

In [ ]:
# Regen WAE
id = 1780
rain = mdb.get_data(id, date_from, date_to)
rain.index = rain.index.tz_localize(None)

# PW Daten
data_pw = pd.read_csv(data_pw, sep=';', date_format='%d.%m.%Y %H:%M',parse_dates=[0])
data_pw.set_index(data_pw.columns[0], inplace=True)
for col in data_pw.columns:
    data_pw.rename(columns={col: col.split(';')[0]}, inplace=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=rain.index, y=rain.iloc[:,0]))

# Analysis PW Data

In [ ]:
df_res = pd.DataFrame()

data_pw = data_pw.loc[:,info_pw['Bezeichnung']]

for name in data_pw.columns:
    pw = data_pw[name]
    pw = pd.DataFrame(pw)

    for event in rain_periods:
        date1 = event[0]
        date2 = event[1]
        rain_period = ts_filter_date(rain, date1, date2)
        pw_period = ts_filter_date(pw, date1, date2)
        pw_period['delta_h'] = pw_period - pw_period.shift()
        pw_period['delta_t'] = pw_period.index
        pw_period['delta_t'] = pw_period['delta_t'] - pw_period['delta_t'].shift()

        mask = pw_period['delta_h'] > 0
        pw_period = pw_period.loc[mask]
        delta_h = pw_period['delta_h'].sum()
        delta_t = pw_period['delta_t'].sum().total_seconds()

        idx = f'{date1} - {date2}'
        df_res.loc[idx, 'Regen [mm]'] = rain_period.sum().values[0]

        A = info_pw[info_pw['Bezeichnung']==pw_period.columns[0]]['Fläche [m2]'].values[0]
        ezg = info_pw[info_pw['Bezeichnung']==pw_period.columns[0]]['Einzugsgebiet [ha]'].values[0]
        V = A * delta_h
        Q_in = V / delta_t * 1000 # l/s

        col = f"{name} [l/s/ha]"
        df_res.loc[idx, col] = Q_in / ezg

In [ ]:
df_res

# Analysis Mike+ Data

In [ ]:
# Read res1d
res1d = Res1D(str(path_res1d))
df_res1d = res1d.read()

In [ ]:
# MUIDS
pws_dict = {
    'PW Klosterweg': {'muids': ['123616', '123620'], 'data': None},
    'PW Mülibach': {'muids': ['123530', '122776'], 'data': None},
    'PW Mülenen': {'muids': ['133878'], 'data': None},
    'PW Schmittenbrunnen': {'muids': ['135106'], 'data': None},
}

In [ ]:
# Read PWs inflows
quantity = 'Discharge:'
for k, v in pws_dict.items():
    muids = v['muids']
    data = get_data_from_muids(df_res1d, quantity, muids)

    # Compute total inflow
    compute_tot_sum(data)
    v['data'] = data

In [ ]:
# Plot the Data
save_dir = Path(r"H:\2 Projekte\8000-\8500er\8532\8532.10 GEP Richterswil\05 Berechnungen Grundlagen\8 Entwässerungskonzept\TS Pumpwerke\Plots\Mike+")
if not save_dir.exists():
    save_dir.mkdir(parents=True)

for k, v in pws_dict.items():
    data = v['data']
    fig = go.Figure()

    for i, col in enumerate(data.columns):
        fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,i], name=col))
    
    fig.update_layout(title=k)

    save_path = save_dir / f'{k}.html'
    fig.write_html(save_path)